# AI Lactate Advisor — Utilities Test Notebook

Validates `model_utils.py` and your trained models. Saves outputs to `/content/test_results.csv`.


In [0]:
import os, sys, numpy as np, pandas as pd
from pathlib import Path
repo_root = Path.cwd()
if str(repo_root) not in sys.path:
    sys.path.append(str(repo_root))
print('Working dir:', repo_root)
print('Files here:', [p.name for p in list(repo_root.iterdir())[:10]])

In [0]:
csv_path = Path('athlete_training_dataset_with_biomarkers.csv')
if csv_path.exists():
    print(f'📂 Found dataset: {csv_path}')
    df = pd.read_csv(csv_path)
else:
    print('⚙️ No dataset found. Generating synthetic test data...')
    n = 300
    df = pd.DataFrame({
        'time': np.arange(n),
        'heart_rate': np.linspace(120, 170, n) + np.random.randn(n)*2,
        'power': np.linspace(200, 280, n) + np.random.randn(n)*5,
        'pace': np.linspace(4.0, 3.2, n) + np.random.randn(n)*0.05
    })
df.head(), df.shape

In [0]:
import importlib
model_utils = importlib.import_module('model_utils')
from model_utils import make_features, prepare_features, smooth_series
try:
    from model_utils import load_model, predict_lactate, predict_recovery, get_shap_summary
except Exception:
    load_model = None; predict_lactate = None; predict_recovery = None; get_shap_summary = None
print('✅ Imported model_utils')

In [0]:
import joblib
from pathlib import Path
lactate_model = None; recovery_model = None
lp = Path('models/lactate_lightgbm_model.joblib'); rp = Path('models/recovery_lightgbm_model.joblib')
if lp.exists():
    try:
        lactate_model = joblib.load(lp); print('✅ Loaded', lp)
    except Exception as e: print('⚠️ Lactate load failed:', e)
if rp.exists():
    try:
        recovery_model = joblib.load(rp); print('✅ Loaded', rp)
    except Exception as e: print('⚠️ Recovery load failed:', e)
lactate_model, recovery_model

In [0]:
try:
    df_feat = prepare_features(df)
except Exception:
    df_feat = make_features(df.tail(30))
pred_lac = None; pred_rec = None
if lactate_model is not None and callable(globals().get('predict_lactate')):
    try: pred_lac = predict_lactate(lactate_model, df_feat)
    except Exception as e: print('⚠️ predict_lactate failed:', e)
if recovery_model is not None and callable(globals().get('predict_recovery')):
    try: pred_rec = predict_recovery(recovery_model, df_feat)
    except Exception as e: print('⚠️ predict_recovery failed:', e)
out = df_feat.copy()
import numpy as np
if pred_lac is not None: out['pred_lactate'] = np.asarray(pred_lac).reshape(-1)
if pred_rec is not None: out['pred_recovery'] = np.asarray(pred_rec).reshape(-1)
out.head(), out.shape

In [0]:
try:
    import shap
    if lactate_model is not None and hasattr(lactate_model, 'predict'):
        Xs = out.drop(columns=[c for c in out.columns if c.startswith('pred_')]).head(256)
        explainer = shap.TreeExplainer(lactate_model)
        sv = explainer.shap_values(Xs)
        shap.summary_plot(sv, Xs, show=True)
        print('✅ SHAP summary displayed')
    else:
        print('ℹ️ SHAP skipped (no model)')
except Exception as e:
    print('ℹ️ SHAP not available:', e)

In [0]:
save_cols = [c for c in out.columns if ('pred_' in c) or c.endswith('_30s') or c.endswith('_slope') or c.endswith('_mean') or c.endswith('_std')]
small = out[save_cols].copy()
out_path = '/content/test_results.csv'
small.to_csv(out_path, index=False)
print('✅ Saved to', out_path)
small.head()